In [ ]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install unsloth

# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# # We have to check which Torch version for Xformers (2.3 -> 0.0.27)
# from torch import __version__; from packaging.version import Version as V
# xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
# !pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("CUDA Version:", torch.version.cuda)
print("Current Device:", torch.cuda.current_device() if torch.cuda.is_available() else "None")
print("CUDA 사용 가능:", torch.cuda.is_available())

print("Torch CUDA 지원 여부:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("PyTorch 버전:", torch.__version__)


In [ ]:
import wandb

wandb.login(key="")

In [ ]:
from unsloth import FastLanguageModel
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
model_id = "unsloth/Llama-3.1-8B-Instruct"
# model_id = "unsloth/gemma-3-12b-it"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = None,# None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # False for LoRA 16bit
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


In [ ]:
from Utils  import format_dataset, ModelType

# EXAONE, gemma, LLaMA3, Alpaca
train_dataset, test_dataset = format_dataset(
    ModelType.LLAMA,
    train_data_files="../00_Data/KoAlpaca_train.json",
)

# 실행 결과 확인
# print("Train Dataset Example:")
# print(train_dataset["train"][0]["text"])

# if test_dataset:
#     print("\nTest Dataset Example:")
#     print(test_dataset["train"][0]["text"])

from unsloth import standardize_sharegpt
train_dataset = standardize_sharegpt(train_dataset['train'])
# test_dataset = standardize_sharegpt(test_dataset['train'])

In [ ]:
lora_r = 8  # 8, 16, 32, 64, 128 중 선택 가능. 클수록 성능 향상, 작을수록 메모리 절약.
lora_alpha = 16 # 16 일반적으로 LoRA의 효과를 조절하는 파라미터 (값이 크면 LoRA 가중치의 영향 증가)
lora_dropout = 0.05

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_r, 
    lora_alpha = lora_alpha,  
    lora_dropout = lora_dropout,  # 0이면 최적화된 성능을 보장. 작은 값으로 설정하면 과적합 방지 가능.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  
    bias = "none",  # "none"이 최적화된 값. 필요에 따라 "all" 또는 "lora_only"로 변경 가능.

    # Unsloth의 최적화된 VRAM 사용 방식 적용 (최대 30% 절약, 2배 큰 배치 크기 가능)
    use_gradient_checkpointing = "unsloth",  # "unsloth" 또는 True 사용 가능. 메모리 절약 효과.
    random_state = 3407,  # 실험의 일관성을 유지하기 위한 랜덤 시드 값.
    # Rank Stabilized LoRA (RSLoRA) 사용 여부
    use_rslora = False,  # False면 일반 LoRA 사용. True면 RSLoRA 적용 가능 (추가적인 안정성 제공).
    loftq_config = None,  # None이면 사용 안 함. 사용하면 LoftQ 기반 저비트 양자화 적용 가능.
)

---

### Finetuning

하이퍼 파라미터 설정

In [ ]:
epoch = 2 # 전체 데이터셋을 몇 번 반복해서 학습할 것인지
batch_size = 4
gradient_step = 2 
learningrate = 2e-4 # 1e-3 ~ 1e-6 가 일반적인 러닝 레이트 범위 ( 1e-4 에서 시작하는거 추천 )
#7e-4
# 1e-3 (0.001) → 매우 높은 학습률, 빠른 학습 가능하지만 불안정할 수도 있음
# 5e-4 (0.0005) → 비교적 빠른 학습, 안정성도 고려한 값
# 1e-4 (0.0001) → 일반적으로 많이 사용되는 기본값
# 5e-5 (0.00005) → 안정성과 학습 속도 균형이 적절한 값
# 1e-5 (0.00001) → 비교적 낮은 학습률, 정밀한 파인튜닝에 적합
# 5e-6 (0.000005) → 매우 낮은 학습률, 기존 모델을 크게 변경하지 않으면서 미세 조정할 때 유용
# 1e-6 (0.000001) → 극도로 낮은 학습률, 작은 변화만 필요할 때 사용
step = 300 # 최대 학습 스텝

wandb_project = f"finetuning"

outName = f"unsloth-{model_id.split('/')[-1]}-{epoch}-{batch_size}-{gradient_step}-{learningrate}-{lora_r}-{lora_alpha}-{lora_dropout}"
output_dir = f"../99_GitLoss/01_RoLaModels/{outName}"

print(outName)
print(output_dir)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,  # 학습할 모델 (LoRA 적용된 LLaMA 등)
    tokenizer = tokenizer,  # 토크나이저 (문장을 토큰 단위로 변환)

    train_dataset = train_dataset,  # 학습에 사용할 데이터셋
    # eval_dataset=test_dataset, # 검증 데이터

    dataset_text_field = "text",  # 데이터셋에서 텍스트 필드 이름 지정

    max_seq_length = max_seq_length,  # 최대 시퀀스 길이 설정
    dataset_num_proc = 1,  # 데이터 전처리 멀티프로세싱 개수 설정 (속도 최적화)

    packing = False,  # ✅ `True`로 설정하면 짧은 시퀀스를 묶어 학습 속도를 5배 증가 가능!

    # 학습 파라미터 설정
    args = TrainingArguments(
        per_device_train_batch_size = batch_size,  # GPU 당 배치 크기 (메모리에 따라 조절 가능)
        gradient_accumulation_steps = gradient_step,  # 그래디언트 누적 스텝 (메모리 절약용)

        num_train_epochs=epoch, 
        # max_steps = 60,  # 최대 학습 스텝 수

        optim = "adamw_torch",  # ✅ 8bit Adam 옵티마이저 사용 (메모리 절약)

        learning_rate = learningrate,  # 학습률 설정
        lr_scheduler_type = "linear",  # 선형 학습률 스케줄 적용 (학습률이 선형적으로 감소)

        fp16 = not is_bfloat16_supported(),  # GPU 환경에 따라 FP16 사용 여부 자동 선택
        bf16 = is_bfloat16_supported(),  # Ampere 이상 GPU에서는 BF16 사용  (A100, RTX 30/40)은 BF16 사용
    
        weight_decay = 0.01,  # 가중치 감소 (Regularization)

        warmup_ratio=0.1, # 상승곡선
        # warmup_steps = 5,  # 초기 워밍업 스텝 설정

        seed = 3407,  # 랜덤 시드 설정 (재현성 보장)

        evaluation_strategy="steps", # eval_steps마다 평가
        eval_steps=5, # eval 훈련 스텝이 xx번 진행될 때마다 검증 데이터셋 평가

        logging_steps = 2,  # 학습 로그 출력 간격
        output_dir = output_dir,  # 학습 결과 저장 경로

        save_strategy="epoch",
        log_level="debug",

        report_to = "wandb",  # "wandb" 또는 "tensorboard"로 변경 가능
    ),
)

# 이전 실행 종료 (안 하면 새로운 실행이 안 생길 수도 있음)
wandb.finish()

# wandb 사용
wandb.init(
    project= wandb_project,
    name=outName,
    reinit=True,
    config={
        "learning_rate": learningrate,
        "batch_size": batch_size,
        "gradient_accumulation_steps": gradient_step,
        "num_train_epochs": epoch
    }
)

trainer_stats = trainer.train()


질의 응답 2가지 버전 ( 완성결과 한번에 출력, 실시간으로 출력 )

In [ ]:
chat_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
지구에 대해 알려줘?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    chat_prompt
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?.", # instruction
        "", # input
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [ ]:

chat_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

inputs = tokenizer(
[
   chat_prompt
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

모델을 저장하고 불러오는 방법 ( Unsloth방식, Huggingface 방식 )

In [ ]:
# LoRA 어댑터만 저장
saveLoRA_dir = f"{output_dir}/LoRA"
model.save_pretrained(saveLoRA_dir) # Local saving
tokenizer.save_pretrained(saveLoRA_dir)

로라 모델 호출 자동으로 병합되는듯

In [ ]:
from unsloth import FastLanguageModel
loadmodel, loadtokenizer = FastLanguageModel.from_pretrained(
    model_name = saveLoRA_dir, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = None,# None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # False for LoRA 16bit
)
FastLanguageModel.for_inference(loadmodel) # Enable native 2x faster inference


In [ ]:

chat_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
다시 합창 합시다' 처럼 거꾸로 읽어도 같은 문장이 영어에도 있나요? 또한 다른 나라의 언어에도 있는 건가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

inputs = loadtokenizer(
[
   chat_prompt
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(loadtokenizer)
_ = loadmodel.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

LoRA 모델을 float16 또는 4bit로 병합하여 저장

In [ ]:
# Merge to 16bit
if True: loadmodel.save_pretrained_merged(f"{output_dir}/modle", loadtokenizer, save_method = "merged_16bit",)

# Merge to 4bit
if False: model.save_pretrained_merged(f"{output_dir}/modle", tokenizer, save_method = "merged_4bit",)

# Just LoRA adapters
if False: model.save_pretrained_merged(f"{output_dir}/modle", tokenizer, save_method = "lora",)

GGUF / llama.cpp 변환

In [ ]:
output_dir

In [ ]:
# # Save to 8bit Q8_0
# if False: model.save_pretrained_gguf(f"{output_dir}/modle", tokenizer)

# # Save to 16bit GGUF
# if True: model.save_pretrained_gguf(f"{output_dir}/modle", tokenizer, quantization_method = "f16")

# # Save to q4_k_m GGUF
# if False: model.save_pretrained_gguf(f"{output_dir}/gguf", tokenizer, quantization_method = "q4_k_m")

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py ./your path/modle --outtype q8_0
